In [3]:
import copy
import numpy as np

In [10]:
def validate_polygon(polygon_vector):

    #takes np.array

    vector_length = len(polygon_vector)
    if vector_length == 1:
        return True

    
    tmp_polygon_vector = np.array(polygon_vector)
    tmp_polygon_vector= np.append(tmp_polygon_vector, -1)
    

    i=1
    last_element = polygon_vector[0]
    current_element = tmp_polygon_vector[i]
    
        
    while i<vector_length and last_element==current_element-1:
        last_element = current_element 
        i+=1
        current_element = tmp_polygon_vector[i]

    while i<vector_length and last_element==current_element:
        last_element = current_element 
        i+=1
        current_element = tmp_polygon_vector[i]
        

    while i<vector_length and last_element==current_element+1:
        last_element = current_element 
        i+=1
        current_element = tmp_polygon_vector[i]

    return True if current_element==-1 else False


In [11]:
validate_polygon(np.array([1,1,1,3]))

False

In [12]:
def cut_zeroes_from_array(array):

    dezeroed_array=[]
    for i in array:
        if i!=0:
            dezeroed_array.append(i)
    return dezeroed_array
            

In [13]:
def convert_polygon_vector_to_arrays(polygon_vector, value=1):

    #takes dezeroed vector as array
    #rretruns list of arrays
    
    height = len(polygon_vector)
    width = max(polygon_vector)
    
    straight_matrix = np.zeros([height, width])
    # matrix for symmetrical and right-leaning polygon (always exists for validated vector)
    duplicate_adjacent_ix = -1 # if exists, a left-leaning solution will be created later
    duplicate_counter = 0
    duplicate_value = -1
    
    for row in range(height):
        row_length = polygon_vector[row] #number of hexes in row
        
        if duplicate_adjacent_ix == -1 and row != height - 1 and row_length == polygon_vector[row + 1]:
            duplicate_adjacent_ix = row
            duplicate_value = row_length
            
        if row_length == duplicate_value:
            duplicate_counter+=1
            
        for column in range(row_length):
            straight_matrix[row][column] = value

    if duplicate_adjacent_ix >=0:
        llm_width = width+duplicate_counter-1
        left_leaning_matrix  = np.zeros([height, llm_width])
        offset=0
        for row in range(height):
            if row > duplicate_adjacent_ix and row < duplicate_adjacent_ix+duplicate_counter:
                offset+=1
            for column in range(width):
                left_leaning_matrix[row][column+offset] = straight_matrix[row][column]
        return [straight_matrix, left_leaning_matrix]
            
    return [straight_matrix]
        


        
        
    

In [14]:
convert_polygon_vector_to_arrays(np.array([1,2,3,3,3,2]))

[array([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 0.]]),
 array([[1., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0.],
        [1., 1., 1., 0., 0.],
        [0., 1., 1., 1., 0.],
        [0., 0., 1., 1., 1.],
        [0., 0., 1., 1., 0.]])]

In [19]:
def generate_polygon_vectors(polygon_size, board_size):
    
    def helper(remaining, current):
        if len(current)>board_size:
            return
        if remaining == 0 and validate_polygon(np.array(current)) and check_if_vector_fits_on_board(np.array(current), board_size):
            polygon_vectors.append(np.array(current))
            return
        for i in range(1, remaining + 1):
            helper(remaining - i, current + [i])
    
    polygon_vectors = []
    helper(polygon_size, [])
    
    return polygon_vectors

In [32]:
generate_polygon_vectors(10,5)

[array([1, 2, 3, 4]),
 array([2, 3, 3, 2]),
 array([3, 4, 3]),
 array([4, 3, 2, 1])]

In [18]:
def check_if_vector_fits_on_board(vector, board_size):
    max_row_length = np.max(vector)
    max_row_count = np.count_nonzero(vector == max_row_length)
            
    min_row_length = np.min(vector)
    row_length_diff = max_row_length - min_row_length
    return max_row_length + max_row_count-1 <= board_size and (max_row_count + row_length_diff - (board_size-max_row_length) <= (board_size+1)/2)

In [6]:
check_if_vector_fits_on_board(np.array([2,2,2,2]), 5)

np.True_

In [228]:
def expand_to_hex_board(polygon, board_height, board_width, left, top):
    # takes np.array matrix repr of polygon, np.array matrix repr of board, and 2 ints
    polygon_height = len(polygon)
    polygon_width = len(polygon[1])

    assert polygon_height + top <= board_height, 'Invalid vertical offset'
    assert polygon_width + left <= board_height, 'Invalid horizontal offset'

    left_offset = np.zeros([polygon_height, left])
    right_offset = np.zeros([polygon_height, board_width - left - polygon_width])
    top_offset = np.zeros([top, board_width])
    bottom_offset = np.zeros([board_height - top - polygon_height, board_width])
    
    expanded_polygon = np.hstack((left_offset, polygon, right_offset))
    expanded_polygon = np.vstack((top_offset, expanded_polygon, bottom_offset))
    
    return expanded_polygon

In [231]:
expand_to_hex_board(np.eye(3), 7, 7, 4, 3)

array([[0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0.]])

In [342]:
def generate_possible_placements(polygon_size, board_size, value):
    polygon_vectors = generate_polygon_vectors(polygon_size, board_size)
    polygons = []
    board_side = int((board_size+1)/2)
    for vector in polygon_vectors:
        polygons.extend(convert_polygon_vector_to_arrays(vector, value))
    for polygon in polygons:
        polygon_height = np.shape(polygon)[0]
        polygon_width = np.shape(polygon)[1]
        last_max_row = board_size - 1 - np.argmax(np.flip(polygon[:,-1]))
        rows_under_last_max = polygon_height-last_max_row-1
        lowest_row_for_last_max_row = board_size - 1 - max(polygon_width-board_side, rows_under_last_max)


In [166]:
a = np.array([[1,2,3],[3,4,5],[3,4,5]])
np.flip(a[:,-1])

array([5, 5, 3])

In [168]:
3 - 1 - np.argmax(np.flip(a[:,-1]))

np.int64(2)

In [169]:
max(1,3)

3